# Demo 2

## Set up simple single-qubit circuit

In [ ]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

In [ ]:
dev = qml.device("default.qubit", wires=1)

In [ ]:
@qml.qnode(dev)
def rotation(thetas):
    qml.RX(1, wires=0)
    qml.RZ(1, wires=0)
    
    qml.RX(thetas[0], wires=0)
    qml.RY(thetas[1], wires=0)

    return qml.expval(qml.PauliZ(0))

## Use gradient descent optimizer

In [ ]:
opt = qml.GradientDescentOptimizer(stepsize=0.05)

thetas = np.zeros(2)
costs_gds = [rotation(thetas)]
thetas_list_gds = [thetas]

for i in range(300):
    thetas = opt.step(rotation, thetas)
    costs_gds.append(rotation(thetas))
    thetas_list_gds.append(thetas)

## Use quantum natural gradient optimizer

In [ ]:
opt = qml.QNGOptimizer(stepsize=0.05)

thetas = np.zeros(2)
costs_qng = [rotation(thetas)]
thetas_list_qng = [thetas]

for i in range(300):
    thetas = opt.step(rotation, thetas)
    costs_qng.append(rotation(thetas))
    thetas_list_qng.append(thetas)

## Compare performance

In [ ]:
plt.plot(costs_gds, label="Gradient Descent")
plt.plot(costs_qng, label="Quantum Natural Gradient")
plt.xlabel("Steps")
plt.ylabel("Cost")
plt.legend()

### Note: the QNG optimizer evaluates the circuit multiple times per step. Moreover, the stepsize/learning rate for both optimizers may not be optimal!

## Compare paths

In [ ]:
thetas_list_gds = np.array(thetas_list_gds)
thetas_list_qng = np.array(thetas_list_qng)

In [ ]:
theta1s = np.linspace(0, 2.5)
theta2s = np.linspace(0, 1.5)
costs_grid = []

for theta1 in theta1s:
    for theta2 in theta2s:
        cost = rotation([theta1, theta2])
        costs_grid.append(cost)

costs_grid = np.array(costs_grid).reshape((50, 50))

In [ ]:
cmap = plt.get_cmap('Greens')
plt.contourf(theta1s, theta2s, costs_grid.T, cmap=cmap)
plt.plot(thetas_list_gds[:, 0], thetas_list_gds[:, 1], "-o", label="Gradient Descent")
plt.plot(thetas_list_qng[:, 0], thetas_list_qng[:, 1], "-o", label="Quantum Natural Gradient")
plt.xlabel(r"$\theta_{1}$")
plt.ylabel(r"$\theta_{2}$")
plt.legend()